In [64]:
"""
in this notebook we will create a csv file with the most common words found in the playlist names
so that we don't have to compute it every time we need this information
data format: name count
usage: process_mpd(path)
"""
import pandas as pd
import numpy as np
import sys
import json
import re
import collections
import os
import datetime
import csv

ntitles = set()
total_playlists = 0
ntitles = set()
title_histogram = collections.Counter()

quick = False
max_files_for_quick_processing = 1

def process_mpd(path):
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            process_info(mpd_slice['info'])
            for playlist in mpd_slice['playlists']:
                process_playlist(playlist)
            count += 1

            if quick and count > max_files_for_quick_processing:
                break

    #show_summary()
    write_file()


def show_summary():
    print ("number of playlists", total_playlists)
    print ("number of unique normalized titles", len(ntitles))
    print ("top playlist titles")
    for title, count in title_histogram.most_common(20):
        print ("%7d %s" % (count, title))
        
def write_file():
    with open('common_titles.csv', 'w', newline='',encoding="utf-8") as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(["name","count"])
        for title, count in title_histogram.most_common(len(title_histogram)) :
            spamwriter.writerow([title,str(count)])

def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name


def process_playlist(playlist):
    global total_playlists, total_tracks, total_descriptions
    total_playlists += 1

    nname = normalize_name(playlist['name'])
    ntitles.add(nname)
    title_histogram[nname] += 1


def process_info(_):
    pass


if __name__ == '__main__':
    path = sys.argv[1]
    if len(sys.argv) > 2 and sys.argv[2] == '--quick':
        quick = False
    #process_mpd(path)
    process_mpd(os.getcwd()+"\MPD")

In [65]:
len(title_histogram)

12816

In [66]:
pd.read_csv("common_titles.csv").head()

,name,count
0,country,2531
1,chill,1769
2,workout,979
3,rap,856
4,christmas,850
